In [1]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import nn
from torchvision import transforms
from torchvision import utils as vutils
from torch.utils.data import DataLoader

In [2]:
save_dir = "./runs/GAN"
data_dir = "/home/pervinco/Datasets/torch_mnist"

epochs = 100
batch_size = 64
d_lr = 0.0001
g_lr = 0.0001

disc_iter = 5
input_dim = 1
latent_dim = 64
hidden_dim = 256
image_size = 784

num_workers = os.cpu_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [4]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, image_size),
            nn.Tanh()
        )

    def forward(self, z):
        z = self.model(z)
        
        return z

In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5], std=[0.5])])

train_dataset = torchvision.datasets.MNIST(root=data_dir,
                                           train=True,
                                           transform=transform,
                                           download=True)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
def save_fake_images(epoch, G, fixed_noise, channels, height, width, num_images=64):
    with torch.no_grad():  # No gradients required
        # Generate fake images from the fixed noise
        fake_images = G(fixed_noise).detach().cpu()
        # Reshape the images to (batch_size, channels, height, width)
        fake_images = fake_images.view(fake_images.size(0), channels, height, width)
        # Make sure the number of images does not exceed the batch size
        fake_images = fake_images[:min(num_images, fake_images.size(0))]
        # Create a grid of images
        grid = vutils.make_grid(fake_images, padding=2, normalize=True)
        # Convert grid to a numpy array
        np_grid = grid.numpy()
        # Transpose the image array (PyTorch tensors have channel in the first dimension, while matplotlib expects it in the third)
        plt.imshow(np.transpose(np_grid, (1, 2, 0)))

    # Save the current figure
    plt.axis("off")
    plt.title(f"Fake Images at Epoch {epoch}")
    plt.savefig(f"{save_dir}/Epoch_{epoch}_Fake.png")
    plt.close()

In [7]:
D = Discriminator(input_dim=image_size)
G = Generator(latent_dim=latent_dim)

D = D.to(device)
G = G.to(device)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=d_lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=g_lr)

fixed_noise = torch.randn(batch_size, latent_dim, device=device)
for epoch in range(epochs):
    for idx, (images, _) in enumerate(tqdm(train_dataloader, desc="Train", leave=False)):
        real_images = images.reshape(batch_size, -1).to(device)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train the discriminator
        # Initialize gradients for both optimizers
        d_optimizer.zero_grad()
        
        # Forward pass for real images
        real_outputs = D(real_images)
        d_real_loss = criterion(real_outputs, real_labels)

        # Sample noise and generate fake data
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_images = G(z)
        fake_outputs = D(fake_images.detach())
        d_fake_loss = criterion(fake_outputs, fake_labels)

        # Backward pass and optimize
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()

        # Train the generator every k steps
        if (idx+1) % disc_iter == 0:
            # Reset gradients for generator optimizer
            g_optimizer.zero_grad()
            
            # Sample noise and generate fake data
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_images = G(z)
            # We want to fool the discriminator, so we assign real labels
            outputs = D(fake_images)
            g_loss = criterion(outputs, real_labels)

            # Backward pass and optimize
            g_loss.backward()
            g_optimizer.step()

    save_fake_images(epoch+1, G, fixed_noise, input_dim, 28, 28)
    # Print losses and scores
    print(f"Epoch[{epoch}/{epochs}], Step[{idx+1}/{len(train_dataloader)}]")
    print(f"D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}")
    print(f"Real Score: {real_outputs.mean().item():.4f}, Fake Score: {fake_outputs.mean().item():.4f}")

# Save the model checkpoints 
torch.save(G.state_dict(), f'{save_dir}/G.ckpt')
torch.save(D.state_dict(), f'{save_dir}/D.ckpt')

Epoch[0/100], Step[937/937]
D_loss: 0.0057, G_loss: 5.9318
Real Score: 1.0000, Fake Score: 0.0057


Epoch[1/100], Step[937/937]
D_loss: 0.0009, G_loss: 8.1174
Real Score: 1.0000, Fake Score: 0.0008


Epoch[2/100], Step[937/937]
D_loss: 0.0004, G_loss: 9.7543
Real Score: 1.0000, Fake Score: 0.0004


Epoch[3/100], Step[937/937]
D_loss: 0.0003, G_loss: 10.8637
Real Score: 0.9999, Fake Score: 0.0002


Epoch[4/100], Step[937/937]
D_loss: 0.0075, G_loss: 9.1340
Real Score: 0.9941, Fake Score: 0.0011


...

Epoch[95/100], Step[937/937]
D_loss: 0.0644, G_loss: 5.2233
Real Score: 0.9652, Fake Score: 0.0082


Epoch[96/100], Step[937/937]
D_loss: 0.1017, G_loss: 6.1498
Real Score: 0.9659, Fake Score: 0.0182


Epoch[97/100], Step[937/937]
D_loss: 0.0244, G_loss: 6.0906
Real Score: 0.9920, Fake Score: 0.0145


Epoch[98/100], Step[937/937]
D_loss: 0.4304, G_loss: 11.7591
Real Score: 0.9112, Fake Score: 0.0000


Epoch[99/100], Step[937/937]
D_loss: 0.0555, G_loss: 4.2978
Real Score: 0.9825, Fake Score: 0.0085
